# Conquer CIFAR10 using resnet

In [1]:
import torch
import numpy
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision
import torch.utils
import torch.optim as optim

# define hyper parameter

In [2]:
batch_size=256
epoch=10
lr=0.001
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### define CNN module

In [3]:
resnet50=models.resnet50()
resnet50.fc= nn.Linear(2048, 10)
resnet50.to(device)
        

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# load train,test set

In [4]:
transform=transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])]
)

cifar_train=torchvision.datasets.CIFAR10("./dataset",
                                         train=True,
                                         transform=transform,
                                         download=True)
cifar_test=torchvision.datasets.CIFAR10("./dataset",
                                         train=False,
                                         transform=transform,
                                         download=True)


train_loader=torch.utils.data.DataLoader(cifar_train,shuffle=True,
                                         batch_size=batch_size,drop_last=True)
test_loader=torch.utils.data.DataLoader(cifar_test,shuffle=True,
                                        batch_size=batch_size,drop_last=True)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
loss_func=nn.CrossEntropyLoss()
optimizer=optim.SGD(resnet50.parameters(),lr=lr,momentum=0.9)

# train code

In [ ]:
for epoch in range(epoch):
    
    for i,data in enumerate(train_loader,0):
        
        img,label=data
        x=img.to(device)
        y_=label.to(device)
        
        optimizer.zero_grad()
        y=resnet50(x)
        
        loss=loss_func(y,y_)
        loss.backward()
        optimizer.step()
        
        if i%30==29:
            print(f"epoch: {epoch}, i: {i} ,loss: {loss}")
        
        
        

# test code

In [ ]:


with torch.no_grad():
    correct=0
    total=0
    for i,data in enumerate(test_loader):
        
        x,y_=data
        x=x.to(device)
        y_=y_.to(device)
        
        y=resnet50(x)
        _,y=torch.max(y,1)
        
        total+=y_.size(0)
        correct+=(y==y_).sum().item()
        
        if i%5==4:
            print(f"precision: {(correct/total)*100}")
    
    
    

precision: 10.0
precision: 9.609375
